In [1]:
import pandas as pd
import numpy as np

In [11]:

# leer el archivo Parquet
dfcatalogo = pd.read_parquet('dfcatalogo.parquet', engine='pyarrow')


funcion get max duration

In [12]:
def get_max_duration(year: int, platform: str, duration_type: str) -> dict:
    # Filtrar el dataframe por tipo de contenido, año y plataforma
    filtered_df = dfcatalogo[(dfcatalogo['type'] == 'movie') & 
                            (dfcatalogo['release_year'] == year) & 
                            (dfcatalogo['plataforma'] == platform)]
    
    # Obtener la película con la duración máxima en el tipo especificado
    max_duration = filtered_df[filtered_df['duration_type'] == duration_type]['duration_int'].max()
    max_duration_movie = filtered_df[(filtered_df['duration_int'] == max_duration) &
                                     (filtered_df['duration_type'] == duration_type)]['title'].values
    
    # Verificar que el array de resultados no está vacío antes de intentar acceder a su primer elemento
    if len(max_duration_movie) > 0:
        return {'pelicula': max_duration_movie[0]}
    else:
        return {'pelicula': ''}

In [5]:
# Ejemplo de llamado a la función get_max_duration
year = 2016
platform = 'amazon'
duration_type = 'min'

max_duration_movie = get_max_duration(year, platform, duration_type)
print(f'La película con duración máxima en el año {year} en la plataforma {platform} en {duration_type} es: {max_duration_movie}')

La película con duración máxima en el año 2016 en la plataforma amazon en min es: {'pelicula': 'gentle evening rain'}


funcion get score count

In [6]:
def get_score_count(df, plataforma, scored, anio):
    # Seleccionar solo las películas de la plataforma y el año especificado
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie') &
                   (df['release_year'] == anio)]
    # Filtrar solo las películas con un puntaje mayor al especificado
    peliculas_con_puntaje = peliculas[peliculas['avg_rating'] > scored]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas_con_puntaje)
    return {
        'plataforma': plataforma,
        'cantidad': cantidad,
        'anio': anio,
        'score': scored
    }

In [7]:
get_score_count(dfcatalogo, 'disney+', 1.45, 2014)

{'plataforma': 'disney+', 'cantidad': 34, 'anio': 2014, 'score': 1.45}

funcion get count platform

In [8]:
def get_count_platform(df, plataforma):
    # Seleccionar solo las películas de la plataforma especificada
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie')]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas)
    return {
        'plataforma': plataforma,
        'peliculas': cantidad
    }

In [9]:
get_count_platform(dfcatalogo, 'hulu')

{'plataforma': 'hulu', 'peliculas': 1484}

funcion get actor

In [9]:
def get_actor(platform:str, year:int ):
    df_filtered = dfcatalogo[(dfcatalogo['plataforma'] == platform) & (dfcatalogo['release_year'] == year)]
    actor_count = {}
    for actors in df_filtered['cast']:
        if isinstance(actors, str):
            for actor in actors.split(','):
                actor_count[actor.strip()] = actor_count.get(actor.strip(), 0) + 1
    if len(actor_count) == 0:
        return "No se encontraron actores"
    else:
        return 'Actor que más se repite según plataforma y año:  ' + max(actor_count, key=actor_count.get)

In [17]:
respuesta = get_actor('amazon', 2010)
print(respuesta)

Actor que más se repite según plataforma y año:  cassandra peterson


funcion prod per county

In [14]:
def prod_per_county(tipo: str, pais: str, anio: int):
    # Filtramos el DataFrame según el tipo de contenido, el país y el año
    df_filtered = dfcatalogo[(dfcatalogo['type'] == tipo) & (dfcatalogo['country'] == pais) & (dfcatalogo['release_year'] == anio)]

    # Contamos el número total de elementos del DataFrame filtrado
    total_peliculas = df_filtered.shape[0]

    return {'pais': pais, 
            'anio': anio, 
            'total de contenido': total_peliculas,
            'tipo_contenido': tipo}

In [15]:
respuesta = prod_per_county('tv show', 'france', 2020,)
print(respuesta)

{'pais': 'france', 'anio': 2020, 'total de contenido': 8, 'tipo_contenido': 'tv show'}


funcion get contents

In [16]:
def get_contents(rating: str):
    # Utilizamos value_counts para contar el número de contenidos según el rating de audiencia
    contenido = dfcatalogo['rating'].value_counts().get(rating, 0)

    return {'rating': rating, 'contenido': contenido}

In [17]:
respuesta = get_contents('r')
print(respuesta)

{'rating': 'r', 'contenido': 2154}


In [8]:

# guardar dataframe como archivo Parquet
dfcatalogo.to_parquet('dfcatalogo.parquet')